In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/submission/m.csv
/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [3]:
data=pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/train_data.csv')
test=pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/test_data.csv')

In [4]:
data=pd.DataFrame(data)
test=pd.DataFrame(test)

In [5]:
data['Offer Accepted'].value_counts()

Yes    6994
No     5385
Name: Offer Accepted, dtype: int64

In [6]:
y=data['Offer Accepted']
X=data.drop(['car','Offer Accepted'],axis=1)

In [7]:
test=test.drop(['car'],axis=1)

In [8]:
from sklearn.impute import SimpleImputer
si=SimpleImputer(strategy='most_frequent')

In [9]:
xi=si.fit_transform(X)
t1=si.fit_transform(test)

In [10]:
xi=pd.DataFrame(xi,columns=['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer', 'Restaur_spend_less_than20',
       'Marital Status', 'restaurant type', 'age',
       'Prefer western over chinese', 'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'restuarant_same_direction_house', 'Cooks regularly', 'Customer type',
       'Qualification', 'is foodie', 'no_Take-aways', 'Job/Job Industry',
       'restuarant_opposite_direction_house', 'has Children',
       'visit restaurant with rating (avg)', 'temperature',
       'Restaur_spend_greater_than20', 'Travel Time', 'Climate',
       'drop location', 'Prefer home food'])
t1=pd.DataFrame(t1,columns=['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer', 'Restaur_spend_less_than20',
       'Marital Status', 'restaurant type', 'age',
       'Prefer western over chinese', 'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'restuarant_same_direction_house', 'Cooks regularly', 'Customer type',
       'Qualification', 'is foodie', 'no_Take-aways', 'Job/Job Industry',
       'restuarant_opposite_direction_house', 'has Children',
       'visit restaurant with rating (avg)', 'temperature',
       'Restaur_spend_greater_than20', 'Travel Time', 'Climate',
       'drop location', 'Prefer home food'])

In [11]:
from sklearn.preprocessing import OrdinalEncoder,MinMaxScaler,LabelEncoder
mms=MinMaxScaler()
one=OrdinalEncoder()
le=LabelEncoder()


In [12]:
X1=mms.fit_transform(xi[['temperature','Travel Time']])
T1=mms.fit_transform(t1[['temperature','Travel Time']])

In [13]:
X2=one.fit_transform(xi[['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer', 'Restaur_spend_less_than20',
       'Marital Status', 'restaurant type', 'age',
       'Prefer western over chinese', 'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'restuarant_same_direction_house', 'Cooks regularly', 'Customer type',
       'Qualification', 'is foodie', 'no_Take-aways', 'Job/Job Industry',
       'restuarant_opposite_direction_house', 'has Children',
       'visit restaurant with rating (avg)',
       'Restaur_spend_greater_than20', 'Climate',
       'drop location', 'Prefer home food']])
T2=one.fit_transform(t1[['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer', 'Restaur_spend_less_than20',
       'Marital Status', 'restaurant type', 'age',
       'Prefer western over chinese', 'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'restuarant_same_direction_house', 'Cooks regularly', 'Customer type',
       'Qualification', 'is foodie', 'no_Take-aways', 'Job/Job Industry',
       'restuarant_opposite_direction_house', 'has Children',
       'visit restaurant with rating (avg)',
       'Restaur_spend_greater_than20', 'Climate',
       'drop location', 'Prefer home food']])

In [14]:
y_train=le.fit_transform(y)

In [15]:
X_train=np.concatenate((X1,X2),axis=1)
X_test=np.concatenate((T1,T2),axis=1)

In [16]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='log', penalty='l2', alpha=0.001, learning_rate='constant', eta0=0.01, max_iter=20, random_state=32)
sgd.fit(X_train, y_train)
sgd.score(X_train, y_train)

0.5700783585103805

In [17]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)

min_samples_split = [ 3,6,9,12,15,18,21,24]

min_samples_leaf = [1, 2, 4,6,8,10,12]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [18]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3,random_state=42,n_jobs=-1)

rf_random.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8, 10,
                                                             12],
                                        'min_samples_split': [3, 6, 9, 12, 15,
                                                              18, 21, 24],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]},
                   random_state=42)

In [19]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 18,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 80,
 'bootstrap': False}

In [20]:
rfc=RandomForestClassifier(n_estimators=100,min_samples_split=18,min_samples_leaf=4,max_features='sqrt',max_depth=80,bootstrap=False,warm_start=True).fit(X_train,y_train)

In [21]:
rfc.score(X_train,y_train)

0.9490265772679538

In [22]:
predict=rfc.predict(X_test)
predict.shape

(5305,)

In [23]:
predict=np.where(predict==1,"Yes","No")

In [24]:
samp= pd.read_csv("/kaggle/input/marketing-strategy-personalised-offer/sample.csv")
id_ = samp.id
y_pred= predict

In [25]:
df = pd.DataFrame({"id" : id_,"Offer Accepted":predict })

In [26]:
submission=df

In [27]:
submission.rename(columns={'Unnamed: 0':'id','0':'Offer Accepted'},inplace=True)
submission.columns

Index(['id', 'Offer Accepted'], dtype='object')

In [28]:
submission.to_csv('submission.csv',index=False)

In [29]:
submission

,id,Offer Accepted
0,0,Yes
1,1,Yes
2,2,No
3,3,No
4,4,Yes
...,...,...
5300,5300,Yes
5301,5301,Yes
5302,5302,No
5303,5303,Yes
